# 包导入及基本配置

In [1]:
import torch
import numpy as np
import pandas as pd

import gc
import time
from collections import Counter

import matplotlib.pyplot as plt
import lightgbm as lgb
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

tqdm.pandas()

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

seed = 2023
num_boost_round = 5000
verbose_eval = 200
early_stopping_rounds = 100
learning_rate = 0.05
max_depth = -1
num_leaves = 128
subsample_freq = 1
subsample = 0.85
colsample_bytree = 0.5
n_splits = 10
reg_alpha = 0
reg_lambda = 10
n_jobs = 64

# 读取数据并重命名列名

In [2]:
data = pd.read_csv('账户交易信息.csv')
data_static = pd.read_csv('账户静态信息.csv')
data_label = pd.read_csv('训练集标签.csv')
data = data.rename({'jylsxh': '交易流水序号',
                    'zhdh': '账户代号',
                    'dfzh': '对方账号',
                    'jdbz': '借贷标志',
                    'jyje': '交易金额',
                    'jyye': '交易余额',
                    'dfhh': '对方行号',
                    'jyrq': '交易日期',
                    'jysj': '交易时间',
                    'jyqd': '交易渠道',
                    'zydh': '摘要代号',
                    'jdbj': '借贷标记',
                    'zhye': '账户余额',
                    'dfmccd': '对方名称长度'}, axis=1)
data_static =  data_static.rename({'khrq':'开户日期', 
                                   'xb': '性别', 
                                   'khjgdh':'开户机构代号', 
                                   'zhdh': '账户代号'}, axis=1)
data_label = data_label.rename({'zhdh': '账户代号', 
                                'black_flag': '标签'}, axis=1)

In [3]:
data.head()

,交易流水序号,账户代号,对方账号,借贷标记,交易金额,账户余额,对方行号,交易日期,交易时间,交易渠道,摘要代号,对方名称长度
0,5D252156AE9F6B6595A1C56F56D4F91C,86C379D938234BAA,14BEFED1370B730A,0,310.00,57806.83,834E1F06,2020-03-01,00:18:06,E96ED478,4E0CB6FB,45
1,8BB3D82CA8E5F95577CA3E2DF432DF64,8EB373F073727157,FD7F11B33576339B,1,599.99,7099.73,B3D461D4,2020-03-01,00:18:17,621461AF,A3C65C29,6
2,412B7E903BC06882EEB9FB6A484D0773,997DED969A377D40,014F2782648E7FDA,1,4000.00,34448.04,A71C76B8,2020-03-01,00:18:30,621461AF,A3C65C29,6
3,F1122F893AC75DC8751190C67E1C3DB6,8EB373F073727157,129FAF9FD9D03346,1,299.98,7399.71,8A1BC467,2020-03-01,00:19:06,621461AF,A3C65C29,6
4,8BD9575EA55E67D4E99AC43B2A444172,8EB373F073727157,3B9CD92F13274EBA,1,999.96,8399.67,A8DA3378,2020-03-01,00:19:17,091D584F,2618045A,6


In [4]:
data_static.head()

,账户代号,开户日期,开户机构代号,性别,年龄
0,DDF394282B1E1508,2018-04-13,577BCC91,1,25
1,CAE68290A37CC77D,2016-04-02,34ED066D,1,27
2,41E4A8AECE47E5F3,2014-09-28,30BB3825,1,44
3,163C42F2A3FD518E,2010-06-11,34ED066D,1,55
4,6FBFEB03252FDB9F,2015-08-20,D64A340B,0,44


In [5]:
df = pd.merge(data, data_static, on='账户代号', how='left')
df = pd.merge(df, data_label, on='账户代号', how='left')

In [6]:
df.head()

,交易流水序号,账户代号,对方账号,借贷标记,交易金额,账户余额,对方行号,交易日期,交易时间,交易渠道,摘要代号,对方名称长度,开户日期,开户机构代号,性别,年龄,标签
0,5D252156AE9F6B6595A1C56F56D4F91C,86C379D938234BAA,14BEFED1370B730A,0,310.00,57806.83,834E1F06,2020-03-01,00:18:06,E96ED478,4E0CB6FB,45,2014-02-28,37BC2F75,0,28,NaN
1,8BB3D82CA8E5F95577CA3E2DF432DF64,8EB373F073727157,FD7F11B33576339B,1,599.99,7099.73,B3D461D4,2020-03-01,00:18:17,621461AF,A3C65C29,6,2013-12-27,C3992E9A,0,31,NaN
2,412B7E903BC06882EEB9FB6A484D0773,997DED969A377D40,014F2782648E7FDA,1,4000.00,34448.04,A71C76B8,2020-03-01,00:18:30,621461AF,A3C65C29,6,2015-02-14,F3F27A32,0,31,NaN
3,F1122F893AC75DC8751190C67E1C3DB6,8EB373F073727157,129FAF9FD9D03346,1,299.98,7399.71,8A1BC467,2020-03-01,00:19:06,621461AF,A3C65C29,6,2013-12-27,C3992E9A,0,31,NaN
4,8BD9575EA55E67D4E99AC43B2A444172,8EB373F073727157,3B9CD92F13274EBA,1,999.96,8399.67,A8DA3378,2020-03-01,00:19:17,091D584F,2618045A,6,2013-12-27,C3992E9A,0,31,NaN


# 金额特征构建

In [7]:
df['交易金额'] = df['交易金额'].fillna(0.)
df['账户余额'] = df['账户余额'].fillna(0.)

df['退款标记'] = (df['交易金额'] < 0).astype(int)
df['交易金额大于账户余额'] = (df['交易金额'] > df['账户余额']).astype(int)

df['原账户余额'] = df['交易金额'] + df['账户余额']
df['消费比例'] = df['交易金额'] / (df['账户余额']+1)

money_features = []
for f in ['交易金额', '账户余额', '原账户余额']:
    X = df[f]
    df[f'{f}_normalize'] = (X - X.min()) / (X.max() - X.min())
    money_features += [f'{f}_normalize']

money_features += ['消费比例']

# 日期特征构建

In [8]:
f = '交易日期_datetime'
df[f] = pd.to_datetime(df['交易日期'].astype(str), format='%Y-%m-%d')
df['交易日期_month'] = df[f].dt.month
df['交易日期_day'] = df[f].dt.day
df['交易日期_weekofyear'] = df[f].dt.weekofyear
df['交易日期_dayofyear'] = df[f].dt.dayofyear
df['交易日期_dayofweek'] = df[f].dt.dayofweek
df['交易日期_is_wknd'] = df[f].dt.dayofweek // 6
df['交易日期_is_month_start'] = df[f].dt.is_month_start.astype(int)
df['交易日期_is_month_end'] = df[f].dt.is_month_end.astype(int)
df['交易日期_相对天'] = (df[f].max() - df[f]).dt.days

del df[f]

time_features = [f for f in df.columns if f.startswith('交易日期_')]

f = '交易时间_split'
df['交易时间_split'] = df['交易时间'].progress_apply(lambda x:x.split(':'))
df['交易时间_小时'] = df[f].progress_apply(lambda x: int(x[0]))
df['交易时间_分'] = df[f].progress_apply(lambda x: int(x[1]))
df['交易时间_秒'] = df[f].progress_apply(lambda x: int(x[2]))
df['交易时间_总分'] = df[f].progress_apply(lambda x: int(x[0]) * 60 + int(x[1]))
df['交易时间_总秒'] = np.log(df[f].progress_apply(lambda x: int(x[0]) * 3600 + int(x[1])*60 + int(x[2]))+1)
del df[f]
    
time_features += [f for f in df.columns if f.startswith('交易时间_')]

time_features

/tmp/ipykernel_2438234/4066833997.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['交易日期_weekofyear'] = df[f].dt.weekofyear
100%|██████████| 816270/816270 [00:00<00:00, 980069.75it/s] 


['交易日期_month',
 '交易日期_day',
 '交易日期_weekofyear',
 '交易日期_dayofyear',
 '交易日期_dayofweek',
 '交易日期_is_wknd',
 '交易日期_is_month_start',
 '交易日期_is_month_end',
 '交易日期_相对天',
 '交易时间_小时',
 '交易时间_分',
 '交易时间_秒',
 '交易时间_总分',
 '交易时间_总秒']

In [9]:
group_features = []
for f in tqdm(['账户代号', '对方账号', '借贷标记', '对方行号',
               '交易渠道', '对方名称长度', '开户机构代号', '摘要代号', 
               '性别', '年龄'] + ['交易时间_小时', '交易时间_分', '交易时间_秒'], desc='频次统计与交叉'):

    tmp = pd.DataFrame()
    value_counts = df[f].map(df[f].value_counts())
    tmp[f'{f}_freq'] = value_counts
    group_features += [f'{f}_freq']

    for ff in money_features:
        tmp[f'{f}_{ff}_freq'] = (df[ff] / value_counts).fillna(0)
        group_features += [f'{f}_{ff}_freq']
    df = pd.concat([df, tmp], axis=1)

频次统计与交叉: 100%|██████████| 13/13 [00:01<00:00,  6.70it/s]


In [10]:
user = pd.merge(data_static, data_label, on='账户代号', how='left')
user.head()

,账户代号,开户日期,开户机构代号,性别,年龄,标签
0,DDF394282B1E1508,2018-04-13,577BCC91,1,25,NaN
1,CAE68290A37CC77D,2016-04-02,34ED066D,1,27,NaN
2,41E4A8AECE47E5F3,2014-09-28,30BB3825,1,44,NaN
3,163C42F2A3FD518E,2010-06-11,34ED066D,1,55,0.0
4,6FBFEB03252FDB9F,2015-08-20,D64A340B,0,44,NaN


In [11]:
f = '账户代号'

for ff in ['借贷标记', '退款标记']:
    for i in [0, 1]:
        for fff in tqdm(group_features, desc=f'Groupby: {ff}={i}'):
            name = ['mean', 'median', 'max', 'std']
            agg = ['mean', 'median', 'max', 'std']
            columns = [f] + [f'{ff}_{fff}_{x}{i}' for x in name]
            tmp = df[df[ff]==i].groupby(f)[fff].agg(agg).reset_index().fillna(0)
            tmp.columns = columns
            user = user.merge(tmp, how='left', on=f)

        for fff in tqdm(time_features, desc=f'Groupby: {ff}={i}'):
            name = ['mean', 'min', 'max', 'ptp']
            agg = ['mean', 'min', 'max', np.ptp]
            columns = [f] + [f'{ff}_{fff}_{x}{i}' for x in name]
            tmp = df[df[ff]==i].groupby(f)[fff].agg(agg).reset_index().fillna(0)
            tmp.columns = columns
            user = user.merge(tmp, how='left', on=f)         

Groupby: 退款标记=1: 100%|██████████| 14/14 [00:01<00:00,  8.38it/s]


In [12]:
tmp = pd.DataFrame()

for f in ['账户代号']:
    # 操作次数
    tmp[f'{f}_操作次数'] = user[f].map(df[f].value_counts()) 

    for ff in ['借贷标记', '退款标记']:
        # 转入次数、转出次数、转入次数占比
        tmp[f'{f}_{ff}_转入次数'] = user[f].map(df.groupby(f)[ff].apply(sum))
        tmp[f'{f}_{ff}_转出次数'] = tmp[f'{f}_操作次数'] - tmp[f'{f}_{ff}_转入次数']
        tmp[f'{f}_{ff}_比例'] = tmp[f'{f}_{ff}_转入次数'] / tmp[f'{f}_操作次数']    
        
        # 转入金额、转出金额、转入金额占比
        tmp[f'{f}_{ff}_转入金额'] = user[f].map(df[df[ff]==1].groupby(f)['交易金额'].apply(sum))
        tmp[f'{f}_{ff}_转出金额'] = user[f].map(df[df[ff]==0].groupby(f)['交易金额'].apply(sum))
        tmp[f'{f}_{ff}_转入金额比例'] = tmp[f'{f}_{ff}_转入金额'] / (tmp[f'{f}_{ff}_转入金额'] + tmp[f'{f}_{ff}_转出金额'])

        # 转入人数、转出人数、转入人数占比
        tmp[f'{f}_{ff}_转入人数'] = user[f].map(df[df[ff]==1].groupby(f)['对方账号'].apply('nunique'))
        tmp[f'{f}_{ff}_转出人数'] = user[f].map(df[df[ff]==0].groupby(f)['对方账号'].apply('nunique'))
        tmp[f'{f}_{ff}_转入人数占比'] = tmp[f'{f}_{ff}_转入人数'] / (tmp[f'{f}_{ff}_转入人数'] + tmp[f'{f}_{ff}_转出人数'])
        
    # for ff in ['交易日期', '交易日期_day', '交易日期_dayofweek']:
    for ff in ['交易日期']:
        # 转账日期数、比例、差
        tmp[f'{f}_{ff}数'] = user[f].map(df.groupby(f)[ff].apply('nunique'))
        tmp[f'{f}_{ff}比例'] = tmp[f'{f}_{ff}数'] / tmp[f'{f}_操作次数'] # 强特
        tmp[f'{f}_{ff}差'] = tmp[f'{f}_操作次数'] - tmp[f'{f}_{ff}数']
        tmp[f'{f}_{ff}差比'] = tmp[f'{f}_{ff}差'] / tmp[f'{f}_{ff}数']

user['开户年'] = LabelEncoder().fit_transform(user['开户日期'].apply(lambda x: x.split('-')[0]))
user['开户月'] = LabelEncoder().fit_transform(user['开户日期'].apply(lambda x: x.split('-')[1]))
user['开户机构代号'] = LabelEncoder().fit_transform(user['开户机构代号'])        
user['开户日期'] = LabelEncoder().fit_transform(user['开户日期'])        
user['年龄'] = LabelEncoder().fit_transform(user['年龄'])    
user = pd.concat([user, tmp], axis=1)

/tmp/ipykernel_2438234/3585167708.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  user['开户年'] = LabelEncoder().fit_transform(user['开户日期'].apply(lambda x: x.split('-')[0]))
/tmp/ipykernel_2438234/3585167708.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  user['开户月'] = LabelEncoder().fit_transform(user['开户日期'].apply(lambda x: x.split('-')[1]))


# TF-IDF特征构建

In [13]:
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec

def svd(x, k=100):
    U, S, V = sp.linalg.svds(x, k=k)
    return U * S


def get_tfidf_feature(text, k=100): # 100
    word_vectorizer = TfidfVectorizer(
        sublinear_tf=True,
        strip_accents='unicode',
        analyzer='word',
        token_pattern=r'\w{1,}',
        stop_words=[',', ':', ' ', '.', '-'],
        ngram_range=(1, 1),
        max_features=1000)

    tfidf = word_vectorizer.fit_transform(text).tocsr().astype('float')
    print('Before:', tfidf.shape)
    if k:
        tfidf = svd(tfidf, k=k)
    else:
        tfidf = tfidf.A
    print('After:', tfidf.shape)
    return tfidf

def get_cnt_feature(text, k=100): # 100
    word_vectorizer = CountVectorizer(analyzer='word', 
                                      ngram_range=(1, 1), max_df=1.0, min_df=1,
                                      token_pattern=r'\w{1,}',
                                      strip_accents='unicode',
                                      stop_words=[',', ':', ' ', '.', '-'],
                                      max_features=1000)

    cnt_vec = word_vectorizer.fit_transform(text).tocsr().astype('float')
    print('Before:', cnt_vec.shape)
    if k:
        cnt_vec = svd(cnt_vec, k=k)
    else:
        cnt_vec = cnt_vec.A
    print('After:', cnt_vec.shape)
    return cnt_vec

df = df.sort_values('交易日期')

for f, k in zip(['对方账号', '对方行号', '开户机构代号', '摘要代号', '交易渠道'], [100, 100, 100, 100, None]):
    tokens = user['账户代号'].map(df.groupby('账户代号')[f].apply(lambda x: '-'.join(list(x)))).values
    emb = get_tfidf_feature(tokens, k=k)
    emb = pd.DataFrame(emb, columns=[f'{f}0_{i}' for i in range(emb.shape[1])])
    user = pd.concat([user, emb], axis=1)
    
    # print(f'CountVec: {f}, K={k}')
    # emb = get_cnt_feature(tokens, k=k)
    # emb = pd.DataFrame(emb, columns=[f'{f}1_{i}' for i in range(emb.shape[1])])
    # user = pd.concat([user, emb], axis=1)    

Before: (6000, 1000)
After: (6000, 100)
Before: (6000, 1000)
After: (6000, 100)
Before: (6000, 425)
After: (6000, 100)
Before: (6000, 158)
After: (6000, 100)
Before: (6000, 21)
After: (6000, 21)


In [14]:
ycol = '标签'
drop_list = ['账户代号', '开户日期'] + [ycol]
categorical_feature = ['性别', '年龄', '开户机构代号', '开户年', '开户月']

features = [c for c in user.columns if c not in drop_list]
assert len(features) - len(set(features)) == 0, '有重复特征！'
print(f'特征数目总计：{len(features)}')

for i in categorical_feature:
    if i in features:
        user[i] = user[i].astype('category')
        
categorical_feature        

特征数目总计：1713


['性别', '年龄', '开户机构代号', '开户年', '开户月']

In [15]:
train = user[user['标签'].notnull()].reset_index(drop=True)
test = user[user['标签'].isnull()].reset_index(drop=True)

# LGB训练

In [16]:
def model_lgb_training(train, suff='SKF0'):
    if suff.startswith('SKF'):
        KF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed+int(suff[-1]))
    elif suff.startswith('KF'):
        KF = KFold(n_splits=n_splits, shuffle=True, random_state=seed+int(suff[-1]))
    else:
        raise RuntimeError
        
    params_lgb = {
        'boosting': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'force_row_wise': True,
        'is_unbalance': True,
        'verbose': -1,
        'random_state': seed,
        'learning_rate': learning_rate,
        'max_depth': max_depth,
        'num_leaves': num_leaves,
        'subsample': subsample,
        'subsample_freq': subsample_freq,
        'colsample_bytree': colsample_bytree,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda,
        'n_jobs': n_jobs,
    }

    oof_lgb = np.zeros(len(train))
    df_importance_list = []
    t = 0

    for fold, (trn_idx, val_idx) in enumerate(KF.split(train, train[ycol])):
        t0 = time.time()
        print(f'-------------------- lgb fold {fold+1} --------------------')

        trn_data = lgb.Dataset(
            train.iloc[trn_idx][features], label=train.iloc[trn_idx][ycol])
        val_data = lgb.Dataset(
            train.iloc[val_idx][features], label=train.iloc[val_idx][ycol], reference=trn_data)

        clf_lgb = lgb.train(
            params=params_lgb,
            train_set=trn_data,
            valid_sets=[trn_data, val_data],
            valid_names=('train', 'val'),
            num_boost_round=num_boost_round,
            categorical_feature=categorical_feature,
            callbacks=[
                lgb.early_stopping(early_stopping_rounds,
                                   first_metric_only=True),
                lgb.log_evaluation(verbose_eval),
            ],
        )

        val_output = clf_lgb.predict(
            train.iloc[val_idx][features], num_iteration=clf_lgb.best_iteration)

        oof_lgb[val_idx] = val_output

        df_importance = pd.DataFrame({
            'column': features,
            'importance_split': clf_lgb.feature_importance(importance_type='split'),
            'importance_gain': clf_lgb.feature_importance(importance_type='gain'),
        })
        df_importance_list.append(df_importance)

        clf_lgb.save_model(f'models/{suff}_lgb_{fold}.txt')
        del trn_data, val_data, clf_lgb
        gc.collect()
        t1 = time.time()
        t += t1 - t0
        print(f"It takes {t1-t0} seconds to train fold {fold+1}\n")

    valid_auc = roc_auc_score(train[ycol], oof_lgb)

    df_importance = pd.concat(df_importance_list)
    df_importance = df_importance.groupby('column').agg('mean').reset_index()
    df_importance = df_importance.sort_values('importance_gain', ascending=False)

    print(f"It takes {t} seconds to train model\n")

    print('模型训练完成！')
    print(f"K 折 valid-auc: {valid_auc:.6f}")
    print("特征重要性：")
    print('='*50)
    print(df_importance.head(30))
    print('='*50)
    return oof_lgb

# 两个种子两种划分方式，十折，一共40个模型
模型文件放在`/models`文件夹里，需要手动新建（懒得写了）

In [17]:
oof_lgb = 0
cnt = 0
suffs = ['KF0', 'KF1', 'SKF0', 'SKF1']
# suffs = ['SKF0']
for suff in suffs:
    oof_lgb += model_lgb_training(train, suff=suff)
    cnt += 1
oof_lgb /= cnt

-------------------- lgb fold 1 --------------------


/home/jtli/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['年龄', '开户年', '开户月', '开户机构代号', '性别']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/jtli/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/jtli/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Training until validation scores don't improve for 100 rounds
[200]	train's auc: 1	val's auc: 0.961902
[400]	train's auc: 1	val's auc: 0.964181
Early stopping, best iteration is:
[337]	train's auc: 1	val's auc: 0.964832
Evaluated only: auc
It takes 12.70993947982788 seconds to train fold 1

-------------------- lgb fold 2 --------------------


/home/jtli/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['年龄', '开户年', '开户月', '开户机构代号', '性别']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/jtli/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/jtli/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[99]	train's auc: 0.999937	val's auc: 0.967676
Evaluated only: auc
It takes 5.7808146476745605 seconds to train fold 2

-------------------- lgb fold 3 --------------------


...

Training until validation scores don't improve for 100 rounds
[200]	train's auc: 1	val's auc: 0.986296
[400]	train's auc: 1	val's auc: 0.99
Early stopping, best iteration is:
[373]	train's auc: 1	val's auc: 0.99
Evaluated only: auc
It takes 13.22494912147522 seconds to train fold 8

-------------------- lgb fold 9 --------------------


/home/jtli/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['年龄', '开户年', '开户月', '开户机构代号', '性别']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/jtli/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/jtli/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[10]	train's auc: 0.996269	val's auc: 0.899074
Evaluated only: auc
It takes 3.10898756980896 seconds to train fold 9

-------------------- lgb fold 10 --------------------


/home/jtli/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['年龄', '开户年', '开户月', '开户机构代号', '性别']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/jtli/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/jtli/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[58]	train's auc: 0.999575	val's auc: 0.97
Evaluated only: auc
It takes 4.517043828964233 seconds to train fold 10

It takes 87.10782384872437 seconds to train model

模型训练完成！
K 折 valid-auc: 0.965531
特征重要性：
                                       column  importance_split  \
438                  借贷标记_对方账号_消费比例_freq_max1              30.1   
1103                 退款标记_交易日期_dayofyear_ptp0              23.1   
1059                              账户代号_交易日期差比              41.3   
1061                              账户代号_交易日期比例              41.5   
749                                  对方行号0_96              49.3   
444                  借贷标记_对方账号_消费比例_freq_std1              30.4   
1151                       退款标记_交易日期_相对天_ptp0              10.9   
1063                           账户代号_借贷标记_转入人数              23.8   
849                                  对方账号0_96              17.2   
586     借贷标记_摘要代号_交易金额_norm

# 测试集预测 

In [18]:
cnt = 0
prediction = np.zeros(test.shape[0])
for suff in suffs:
    for i in range(n_splits):
        clf_lgb = lgb.Booster(model_file=f'models/{suff}_lgb_{i}.txt')
        cnt += 1
        prediction += clf_lgb.predict(test[features])
prediction /= cnt    

# 输出预测结果 

In [19]:
best_thresh = 0.5
acc = f1_score(train[ycol], np.where(oof_lgb > best_thresh, 1, 0), average='macro')
print(f'训练集的f1: {acc}')
test['标签'] = np.where(prediction > best_thresh, 1, 0)
test[['账户代号', '标签']].rename({'账户代号': 'zhdh', '标签': 'black_flag'}, axis=1).to_csv('result.csv')
test['标签'].describe()

训练集的f1: 0.9146626483793592


count    4800.000000
mean        0.226042
std         0.418310
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: 标签, dtype: float64